In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import numpy as np
import pickle
from PIL import Image
from sentence_transformers import SentenceTransformer, util

In [3]:
data_dir = '/sda/rina_1921cs13/Nischal/NovFake/data/'
text_arr = np.load(os.path.join(data_dir, 'text_array.npy'))
labels_arr = np.load(os.path.join(data_dir, 'labels.npy')).squeeze() # 0 fake, 1 real
ids_arr = np.load(os.path.join(data_dir, 'ids.npy')).squeeze()

In [4]:
image_arr = np.load(os.path.join(data_dir, 'image_array.npy')).squeeze()

In [5]:
num_images, sources, width, height, num_channels = image_arr.shape
img_data_reshape = np.reshape(image_arr, newshape=(num_images, sources, num_channels, width, height))
img_data_target = img_data_reshape[:,0,:,:,:] # Don't convert to GPU
print('New Target Shape', img_data_target.shape)

New Target Shape (11766, 3, 224, 224)


# Embeddings

In [6]:
# Clip (Image/Text Model)
model_img = SentenceTransformer('clip-ViT-B-32')

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


In [9]:
sample_img_embed = model_img.encode(Image.fromarray(img_data_target[0].astype('uint8'), 'RGB'))
sample_txt_embed = model_img.encode(text_arr[0][0])
sample_comb_embed = np.concatenate([sample_img_embed, sample_txt_embed])
print(type(sample_img_embed))
print(sample_img_embed.shape)
print(sample_txt_embed.shape)
print(sample_comb_embed.shape)

<class 'numpy.ndarray'>
(512,)
(512,)
(1024,)


In [24]:
#NOTE: Get all embeddings
all_embeds = []
for i in range(len(text_arr)):
    if i%1000==0:
        print(i)
    img_embed = model_img.encode(Image.fromarray(img_data_target[i].astype('uint8'), 'RGB'))
    try:
        txt_embed = model_img.encode(text_arr[i][0])
    except:
        txt_embed = model_img.encode(text_arr[1671][0][0:380])
    comb_embed = np.concatenate([img_embed, txt_embed])
    all_embeds.append(comb_embed)
embed_arr = np.array(all_embeds)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [25]:
#save
np.save('../Data/clip_embed', embed_arr)

# Clustering

In [26]:
# Fitting Clustering models
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=43).fit(embed_arr)

In [28]:
from collections import Counter
cluster_labels = kmeans.labels_
cluster_counts = dict(Counter(cluster_labels.tolist()))
print(cluster_counts)

{1: 570, 6: 950, 3: 1092, 7: 452, 2: 1778, 0: 2034, 8: 1244, 9: 607, 5: 1544, 4: 1495}


In [29]:
# Save cluster labels
np.save('../Data/nis_event_labels', cluster_labels)